In [1]:
import pandas as pd
import numpy as np

from IPython.display import display, display_html

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

from simulator import *
from simulator_plotting import *

init_notebook_mode(connected=True)

In [9]:
#we seem to have lost some stochasticity?

landscape1 = dataset2.loc['AMC'].tolist()
landscape2 = dataset2.loc['AM'].tolist()

plot_simulation(simulate([landscape1, landscape2], durations=[200, 1000], prob_mutation=1.0e-8),genotype_strings(16))

In [5]:
landscape = dataset1[2].iloc[5].tolist()
plot_simulation(simulate(landscape))

In [7]:
landscape = dataset2.loc['AMC'].tolist()
landscape2 = dataset2.loc['AM'].tolist()
count = 0
for i in range (0, 100):
    results = simulate([landscape, landscape2], durations=[200, 1000], prob_mutation=1.0e-8)
    if results['T_1'] != -1:
        #lucky_results = results
        #plot_simulation(lucky_results, genotype_strings(16))
        count = count + 1
count

61

In [147]:
lucky_results['actual_path']

['0100', '0000', '0001', '1001', '1101']

In [2]:
landscape = dataset2.loc['AMC'].tolist()
landscape2 = dataset2.loc['AM'].tolist()
results = simulate([landscape, landscape2], frequency=200)
plot_simulation(results)

## Which trajectory does a population follow?

The **greedy path** can be determined by applying a simple algorithm to the fitness landscape. Start at the seed genotype and choose its fittest (highest growth rate) mutational neighbor. Continue choosing fittest mutational neighbors until the global optimum is reached, or until a mutational neighbor with higher fitness than the current genotype can no longer be found. In the latter case, a local optimum has been reached.

Through modifications to the simulator code, an "**actual path**" can be determined. During the mutation phase of the simulation, determine whether each mutant indicates the first appearance of a genotype. If a genotype is appearing for the first time, store the genotype it mutated from in an array. After the simulation has finished running, generate a path by starting from the dominant genotype and adding the genotype it mutated from, and then the genotype that genotype mutated from, and so on, until the seed is reached.

Often, during a simulation the population will simply follow the trajectory of the greedy path. Sometimes a path leading to a local optimum will out-compete the greedy path, and so the "actual path" represents the true evolutionary trajectory of the population. Other times the greedy path and the "actual path" will differ, but both lead to the global optimum. In this case it can be said that the true trajectory of the population consists of the greedy path and the "actual path" being followed in parallel. Since both paths are being followed, re-running the simulation multiple times will likely lead to some proportion of the resulting "actual paths" being equivalent to the greedy path.

In sum, plotting the greedy path and the "actual path" will likely suffice in getting a sense of the evolutionary trajectory of the population.

## The effects of population size on switching

By default we have been running simulations with the carrying capacity (and initial population of the seed genotype) set to 10^9.

In [244]:
landscape1 = dataset2.loc['AMC'].tolist()
landscape2 = dataset2.loc['AM'].tolist()
plot_simulation(simulate([landscape1, landscape2], frequency=50))

In [253]:
landscape1 = dataset2.loc['AMC'].tolist()
landscape2 = dataset2.loc['AM'].tolist()
plot_simulation(simulate([landscape1, landscape2], frequency=25, carrying_cap=int(1.0e4), prob_mutation=1.0e-3))

In [3]:
landscape = dataset2.loc['CRO'].tolist()
results = simulate(landscape, prob_mutation=1.0e-7)
plot_simulation(results, genotype_strings(16))
results['num_loci']

4